# Sales Predictive Modeling: Exploratory Data Analysis

In [1]:
import numpy as np
import pandas as pd
import pandas.api.types as ptype
import matplotlib.pyplot as plt
import seaborn as sns

## Approach for EDA: 

**Exploratory Data Analysis (EDA):**
   - **Visualize Data:** Use plots to understand relationships between features and the target variable.
   - **Correlation Analysis:** Identify how features are correlated with sales and with each other.
   - **Feature Importance:** Determine which features are most influential in predicting sales.

## Visualising Relationship between features and target variable

For building an sales predictive model to efficiently learn from sales data stored in BigMart database, it is important to discover relationships and patterns of features through visualisations. 